# Portfolio Opimization Notes

## What is portfolio optimization ?

A **portfolio** is an allocation of funds to a set of stocks. 
**Portfolio optimization** means given a set of assets and a time period we need to find an allocation of funds to those assets that maximizes performance.

To quantify performance, we can choose from a number of metrics like -

1. Cumulative return
2. Average return (mean)
3. Volatility (std or risk)
4. Risk adjusted return (Sharpe ratio)

We can proceed with risk adjusted return becuase it covers both the return and the risk. Higher is the Sharpe ratio the higher is the performance. But if we use our optimizer then it minimizes the error or cost function. We can use it to maximize the Sharpe ratio by taking a negative of Sharpe ratio as the metric.

Minimizing the negative of Sharpe ratio means maximizing the Sharpe ratio.

## Returns and Kurtosis

The distribution of returns for stocks and market indices looks **very similar to Gaussian** but it is not truly Gaussian. This behaviour is observed for all types of returns like daily, weekly, monthly and annual etc. returns.

If those returns were really Gaussian (which they are not) then we would have said that those were normally distributed which means to have a Kurtosis of 3 or Excess Kurtosis of 0. But in reality their **kurtosis is more than 3 which means their excess kurtosis is more than 0**.

Kurtosis is the probability of a value lying in the tails of the distribution. The higher the kurtosis is the more the number of values lying in the tails are and the thicker the tails become.

In many research cases we assume the financial returns to be normally distributed but it can be very dangerous because we ignore their kurtosis. For example, the recession of 2008 due to sub-prime lender crisis when the people committed defaults on their mortgages. The lenders made 2 assumptions both of which were wrong -

1. The returns were assumed to be independent (proved to be wrong after the losses)
2. The returns were assumed to be normally distributed (proved to be wrong after the losses)

## Daily portfolio value

Given that,

```python
start_val = 1000000
start_date = '2009-01-01'
end_date = '2011-12-31'
symbols = ['SPY', 'XOM', 'GOOG', 'GLD']
allocs = [0.4, 0.4, 0.1, 0.1]            # such that the total of allocs is 1.0 i.e., 100%
```

Using this information how to calculate the total portfolio value day by day ?

* We have the `prices` for assets for a period let us say daily prices but it can be any unit period like 1 or 5 minutes.
* We do `normed = prices / prices[0]` where the first row becomes `1.0` and all subsequent prices become cumulative returns.
* Then we do the allocations `alloced = normed * allocs` for each of the asset in the given ratio, this scales the unit period returns as per the allocation ratio for each period.
* Then we calculate the `pos_vals = alloced * start_val` to get the cash allocated to each asset in the portfolio, which gets scaled for each period based on the allocations. Each `pos_val` shows the open position value for each asset over time to show how much it is worth after each unit period.
* We calculate `port_val = pos_vals.sum(axis=1)` which gives the total portfolio value for each period by summing up the value of each individual asset over that period. The `port_val[0]` is same as `start_val` but the `port_val[-1]` gives the final worth of the portfolio on `end_date`.

## Portfolio statistics

Once we have the `port_val` dataframe, we need to compute portfolio statistics on it. From that we get the metric which we need to maximize.

* From `port_val` we calculate the `daily_rets` if we are using daily prices, otherwise we can also call it as `returns`. First row is always `0` as there is no previous period price before the first period. Pandas fills it as `NaN`. We can drop the first row using `daily_rets = daily_rets[1:]` or `daily_rets.dropna(axis=0, inplace=True)` to keep the genuine values.
* We can calculate any of these metrics to compute the portfolio performance -
    + `cum_ret`: How much the value of the portfolio has gone up from the beginning of the period till the end of the period, this is calculated as `(port_val[-1] / port_val[0] ) - 1`.
    + `avg_daily_ret`: Mean return over the entire period calculate as `daily_rets.mean()`.
    + `std_daily_ret`: Standard deviation of returns over the entire period calculate as `daily_rets.std()`.
    + `sharpe_ratio`: Risk adjusted return calculated as `mean(daily_rets) / std(daily_rets)`. If we put the discounted `daily_rf` in that we get `mean(daily_rets - daily_rf) / std(daily_rets - daily_rf)` and we see that `daily_rf` impacts the `mean` but doesn't affect the `std` because deviation of a distribution after adding a constant to each value is still the same, the constant gets cancelled out. So we get the updated formula as `mean(daily_rets - daily_rf) / std(daily_rets)`. Usually we use `sharpe_ratio` as a metric for optimization and consider its maximum possible value such that the return is maximum possible and the risk is minimum possible.
* We convert Sharpe Ratio to annualized value by using `Sharpe_ratio_annual = sharpe_ratio * sqrt(252)` and use it for analysis and comparison of the portfolio performance.

## Mathematical background


$$ \LARGE Sharpe \; Ratio, SR = \frac{\mathbb{E}{[R_p - R_f]}}{\mathbb{S}[R_p - R_f]} = \frac{mean(r_p - r_f)}{std(r_p - r_f)} = \frac{mean(r_p - r_f)}{\sigma_p}  $$


$$ \LARGE Annualized \; Sharpe \; Ratio, SR_{annualized} = SR * K $$

where,

$ \Huge R_p  => $ Overall portfolio return

$ \Huge r_p  => $ Daily portfolio return

$ \Huge R_f => $ Annual risk free rate of return

$ \Huge r_f => $ Daily risk free rate of return

$ \Huge \sigma_p => $ Standard deviation of portfolio return = $ \Huge std(r_p) = std(r_p - r_f) = \sigma_p $ 

$ \Huge \mathbb{E} => $ Mean

$ \Huge \mathbb{S} => $ Standard Deviation

$ \Huge K => $ Adjustment Factor = $ \Huge \sqrt{\# Data \; Samples \; Per \; Year} $

|Time Frame|# Data Samples Per Year|K|
|-|-|-|
|Daily|252|$\sqrt{252}$|
|Weekly|52|$\sqrt{52}$|
|Monthly|12|$\sqrt{12}$|
|Quarterly|4|$\sqrt{4}$|
|Yearly|1|$\sqrt{1}$|

If $ \Huge R_f $ is annual value then we need to get its value for our time period like daily or weekly or monthly etc., that discounted value is $ \Huge r_f $. It can be calculated using discounting-compounding formula -

$ \Huge A = P * {(1 + R)}^{T} $

=> $ \Huge \frac{A}{P} = {(1 + R_f)}^{1} $

If P has grown to A at a risk free rate of $ \Huge R_f $ and we want to discount its value $ \Huge r_f $ on a daily time frame then we need to scale T as well.

$ \Huge R_f $ on daily time frame = $ \Huge r_f $

T = 1 year = 252 trading days

therefore,

$ \Huge (1 + R_f) = {(1 + r_f)}^{252} $

=> $ \Huge (1 + r_f) = {(1 + R_f)}^{\frac{1}{252}} $

=> $ \Huge r_f = \sqrt[\leftroot{-3}\uproot{3}252]{(1 + R_f)} - 1$

This $ \Huge r_f $ is very small and does not change very often, so we approximate it to be $ \Huge 0 $ in the calculations of Sharpe Ratio.

This value of $ \Huge r_f $ is not annualized. It is the value obtained at the end of `end_date` for the given time period say daily, weekly, monthly etc. For each time period it will be different and can not be compared with similar values obtained for differnt time periods. So we always convert it to its annual value called **Annualized Sharpe Ratio**. We multiply Sharpe Ratio with an Adjustment Factor, K to convert it to an annualized value.

For daily time frame we get -

$ \Huge r_{f_{annualized}} = (\sqrt[\leftroot{-3}\uproot{3}252]{(1 + R_f)} - 1) * \sqrt{252} $

**Note: Even if we did not trade for all 252 days in an year, the K is always $ \Huge \sqrt{252} $ becuase of the sampling frequency not because of the total number of days we traded.**


## Basis points

$$ \LARGE 1 \; bps = \frac{1}{100}^{th} \; of \; a \; \%  = \frac{1}{100} * \frac{1}{100} = \frac{1}{10000} = 0.0001 $$

It is called as **bps** or **bips**. We use it to represent those real (fractional) values which are -

* Very small and usually of the order of one-tenth or one-hundredth or etc. of 100
* Are on a smaller time period which could be like a momentary transaction, seconds, minutes, hours, daily etc.

For example,

* If an average daily return of a portfolio is 0.1%, it is actually 0.001 as a multiplier. So instead of calling it as 0.1% or 0.001 we call it 10 bps.
* If a daily risk free rate of return is 0.02%, it is actually 0.0002 as a multiplier. But instead of calling it 0.02% or 0.0002 we call it 2 bps.
* Similarly if the daily risk (std.dev.) of a portfolio is 0.1% which is again 0.001 as as multiplier but we call it as 10 bps.
* Same thing goes for brokerage and slippage in each transaction, we measure those in basis points or bps or bips.

## Framing the optimization problem

In order to maximize the portfolio value we need to maximize a statistical metric that puts more weightage to the stocks with geater returns and/or lesser risk and less or no weightage to the stocks with lower returns and/or greater risk. We have following 3 such metrics -

1. maximize the portfolio return
2. minimize the portfolio volatility (or risk)
3. maximize the sharpe ratio i.e., maximize the $ \large \frac{return}{risk} $ which also means maximizing the return while minimizing the risk

Out of these, the $ 1^{st} $ option is the easiest one to implement because in this approach the 100% weightage will be given to a single stock with maximum returns. In the $ 2^{nd} $ and $ 3^{rd} $ options we will have to consider a combination of stocks, so they will be more complex than the $ 1^{st} $ one.

But even then the $ 3^{rd} $ option is the best because it considers both the $ 1^{st} $ and $ 2^{nd} $ by maximizing the numerator and minimizing the denominator.

In our optimization problem we take a loss or error function and try to minimize it. But here we need to maximize the sharpe ratio metric. So we can consider to minimize the negative of sharpe ratio as a loss or error and use the same optimization process as to minimize that loss.

Following steps can be followed to solve the optimization problem -

1. Provide a function to minimize f(X) => here X is a combination of allocation weights we need that minimizes the loss or error function
2. Provide an initial guess for X => we can take it as equal allocatio weights in the beginning
3. Call the optimizer => which can alter the allocation weights to reach that combination which minimizes the negative sharpe ratio

## Bounds and Constraints

They help - 
* the optimizatin process run faster
* get the right answer

### Bounds/Ranges

These are the limits on values for X in f(X) and are applied individually to each candidate in X. They show the range of values that each candidate in X can take. They are used to represent the valid values for X by excluding the invalid values. This enables the optimization proecess take into account only a smaller set of values thereby leading to a faster convergence.

Here, X represents a weight vector for various assets in the portfolio. Each weight can take value only from 0\% to 100\%, so a valid range for each candidate in X is (0, 1).

Our optimizer will focus on values between 0 and 1 for all allcation weights in the weight vector X.

$$ \LARGE X \in (0, 1) $$

### Constraints

These are the properties of X that must be true and are applied to all candidates in X at once. One such property is that the sum of all asset weights should be equal to 1 i.e., 100\%.

$$ \LARGE \sum_{i=0}^{i=n} x_i = 1 $$

$$ \LARGE \left( \sum_{i=0}^{i=n} x_i \right) - 1 = 0 $$

---